MODEL DEPLOYMENT

GOALS
1. Create a Garch Model class with methods of wrangling data, training, predicting, serving and loading models.
2. Build web API using Fast API and uvicorn.
3. Build data classes for API using pydantic
4. Create API paths for training model and serving predictions.

In [1]:
# Load the SQL extension
%load_ext sql

# Connect to your SQLite database
%sql sqlite:////home/denis/Stock-Data/sqlite.db

import pandas as pd
import sqlite3
import os
from glob import glob
import joblib
from glob import glob
import requests
from data import SQLRepository
from arch.univariate.base import ARCHModelResult

Connect to Database

In [2]:
# Connect to database
connection = sqlite3.connect("/home/denis/Stock-Data/sqlite.db", check_same_thread=False)

# Create an isnstace of the SQLRepository class
repo = SQLRepository(connection=connection)


GARCH MODEL CLASS

In [3]:
from model import GarchModel
from settings import model_directory

# Instantiate a "Garch Model class"
gm_apple = GarchModel(ticker="AAPL", repo=repo, use_new_data=False)

Garch Model Class: Wrangle Method

In [4]:
# Instantiate Garch Model, use new data
model_apple = GarchModel(ticker="AAPL", repo=repo, use_new_data=True)

# Wrange data
model_apple.wrangle_data(n_observations=2000)

model_apple.data.tail()

date
2024-09-26    0.508018
2024-09-27    0.118671
2024-09-30    2.287194
2024-10-01   -2.914163
2024-10-02    0.251978
Name: return, dtype: float64

Garch Model Class: Fit Methood

In [5]:
# Instantiate model, use old data
model_apple = GarchModel(ticker="AAPL", repo=repo, use_new_data=False)

# Wrangle data
model_apple.wrangle_data(n_observations=2500)

# Fit Garch(1, 1) model
model_apple.fit(p=1, q=1)

# Check model paramaters
model_apple.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -5287.88
Distribution:                  Normal   AIC:                           10583.8
Method:            Maximum Likelihood   BIC:                           10607.1
                                        No. Observations:                 2500
Date:                Thu, Oct 03 2024   Df Residuals:                     2499
Time:                        09:52:43   Df Model:                            1
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.0310  7.753e-02      0.399      0.690 [ -0.121,  0.183]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0499      0.186      0.268      0.789     [ -0.315,  0.415]
alpha[1]       0.0649  1.710e-02      3.794  1.484e-04 [3.136e-02,9.839e-02]
beta[1]        0.9351  8.204e-02     11.398  4.257e-30     [  0.774,  1.096]
============================================================================

Covariance estimator: robust
"""

Garch Model Class: Predict Volatility Method

In [6]:
prediction = model_apple.predict_volatility(horizon=5)
prediction

2024-10-03T00:00:00    1.316166
2024-10-04T00:00:00    1.321604
2024-10-07T00:00:00    1.326975
2024-10-08T00:00:00    1.332282
2024-10-09T00:00:00    1.337526
dtype: float64

Garch Model Class: Dump Method

In [7]:
# Save "model_apple" model , asigning filename
filename = model_apple.dump()

Garch Model Class: Load Method

In [8]:
model_apple = GarchModel(ticker="AAPL", repo=repo, use_new_data=False)

# Load model
model_apple.load()

model_apple.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -5287.88
Distribution:                  Normal   AIC:                           10583.8
Method:            Maximum Likelihood   BIC:                           10607.1
                                        No. Observations:                 2500
Date:                Thu, Oct 03 2024   Df Residuals:                     2499
Time:                        09:52:43   Df Model:                            1
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.0310  7.753e-02      0.399      0.690 [ -0.121,  0.183]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0499      0.186      0.268      0.789     [ -0.315,  0.415]
alpha[1]       0.0649  1.710e-02      3.794  1.484e-04 [3.136e-02,9.839e-02]
beta[1]        0.9351  8.204e-02     11.398  4.257e-30     [  0.774,  1.096]
============================================================================

Covariance estimator: robust
"""